In [63]:
import pandas as pd

In [8]:
dry=pd.read_excel("dry_itc_order.xlsx")

In [9]:
box=pd.read_excel("dry_dimentions.xlsx")

In [64]:
box['box volume'] = box.apply(lambda row: (row['Box L'] * row['Box B'] * row['Box H'])/1000, axis=1)
box['item volume'] = box.apply(lambda row: (row['Length'] * row['Breadth'] * row['Height'])/1000, axis=1)

In [65]:
def mu(item,vol):
    if item>0:
        return item*vol
    elif item<0:
        return item*vol*-1

In [66]:
box = box.dropna(subset=["box volume"])

In [67]:
merged_df = pd.merge(dry, box, on="Material Number", how="inner")

In [68]:
merged_df.head()

,Document type,Document type description,Distribution Channel,Plant,Document Date,Billing Date,Sold-To Party,Customer Name,Customer Pin code,Billing Item,...,Height,Weight,Box L,Box B,Box H,Box W,Garland/Pack,Material Volume,box volume,item volume
0,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1014176,AMMANI SUPER MARKET,600063,1,...,35,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25
1,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1001658,FANCY GENERAL STORE,601201,6,...,35,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25
2,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1012713,NEW AZHRUDDIN STORE - TBM,601201,15,...,35,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25
3,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1002483,REJITHA STORE (TBM),600045,1,...,35,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25
4,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1090023,LAKSHMI STORE P PATHAI,601201,13,...,35,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25


In [69]:
merged_df['tot item volume'] = merged_df.apply(lambda row: mu(row['Billing QTY'], row['item volume']), axis=1)


In [70]:
merged_df.head()

,Document type,Document type description,Distribution Channel,Plant,Document Date,Billing Date,Sold-To Party,Customer Name,Customer Pin code,Billing Item,...,Weight,Box L,Box B,Box H,Box W,Garland/Pack,Material Volume,box volume,item volume,tot item volume
0,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1014176,AMMANI SUPER MARKET,600063,1,...,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25,1155.0
1,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1001658,FANCY GENERAL STORE,601201,6,...,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25,1155.0
2,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1012713,NEW AZHRUDDIN STORE - TBM,601201,15,...,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25,2310.0
3,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1002483,REJITHA STORE (TBM),600045,1,...,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25,1155.0
4,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1090023,LAKSHMI STORE P PATHAI,601201,13,...,35.0,275.0,220.0,325.0,5.0,24.0,96250,19662.5,96.25,1155.0


In [71]:
def wasted(so_quantity, size):
    if size==0:
        return 100
    a=so_quantity%size
    if a==0:
        return a
    else:
        return size-a

In [72]:
import math

def count_boxes(item_volume, box_volume):
    boxes_needed = math.ceil(item_volume / box_volume)
    return boxes_needed


In [73]:
def cost(l, b, h):
    v = l * b * h
    merged_df[f'L{l}B{b}H{h}'] = merged_df.apply(lambda row: wasted(row['tot item volume'], v)*0.00239692312 + count_boxes(row['tot item volume'], v)*2.0834 , axis=1)


In [74]:
length=[61,36,38,54,48,42,31,61,52,48,37,40,31,38,36,33,32,28,49,34,30,52]
breadth=[40,25,34,35,34,33,21,41,28,28,27,20,22,27,28,25,28,22,34,24,21,31]
height=[15,36,38,50,25,31,31,50,24,26,26,29,31,31,30,38,26,33,23,30,31,22]

merged_dry_box = []
waste = 0
count = 0
for i in range(len(length)):
    l = length[i]
    b = breadth[i]
    h = height[i]
    cost(l,b,h)

In [75]:
merged_df

,Document type,Document type description,Distribution Channel,Plant,Document Date,Billing Date,Sold-To Party,Customer Name,Customer Pin code,Billing Item,...,L31B22H31,L38B27H31,L36B28H30,L33B25H38,L32B28H26,L28B22H33,L49B34H23,L34B24H30,L30B21H31,L52B31H22
0,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1014176,AMMANI SUPER MARKET,600063,1,...,49.990702,75.551491,71.797909,74.458494,55.153675,48.039607,91.160254,57.991632,46.126862,84.319435
1,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1001658,FANCY GENERAL STORE,601201,6,...,49.990702,75.551491,71.797909,74.458494,55.153675,48.039607,91.160254,57.991632,46.126862,84.319435
2,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1012713,NEW AZHRUDDIN STORE - TBM,601201,15,...,47.222256,72.783044,69.029463,71.690047,52.385229,45.271161,88.391808,55.223186,43.358416,81.550989
3,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1002483,REJITHA STORE (TBM),600045,1,...,49.990702,75.551491,71.797909,74.458494,55.153675,48.039607,91.160254,57.991632,46.126862,84.319435
4,ZREB,WC Brand Returns,58,1023,01-06-2023,01-06-2023,1090023,LAKSHMI STORE P PATHAI,601201,13,...,49.990702,75.551491,71.797909,74.458494,55.153675,48.039607,91.160254,57.991632,46.126862,84.319435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20404,ZBRD,WC Branded Sales,58,1023,22-06-2023,22-06-2023,1087834,ANANDHA STORE,601201,220,...,47.298658,72.859446,69.105865,71.766449,52.461631,45.347563,88.468210,55.299587,43.434818,81.627391
20405,ZBRD,WC Branded Sales,58,1023,23-06-2023,23-06-2023,1002768,ANNAI STORE(TBM),600045,60,...,46.934625,72.495414,68.741832,71.402417,52.097598,44.983530,88.104177,54.935555,43.070785,81.263358
20406,ZBRD,WC Branded Sales,58,1023,23-06-2023,23-06-2023,1601795,ANNDERSON TEA STALL,600034,20,...,46.934625,72.495414,68.741832,71.402417,52.097598,44.983530,88.104177,54.935555,43.070785,81.263358
20407,ZBRD,WC Branded Sales,58,1023,24-06-2023,24-06-2023,1093181,SHANTHI COOL BAR,600029,20,...,46.934625,72.495414,68.741832,71.402417,52.097598,44.983530,88.104177,54.935555,43.070785,81.263358


In [76]:
a=[]
length=[61,36,38,54,48,42,31,61,52,48,37,40,31,38,36,33,32,28,49,34,30,52]
breadth=[40,25,34,35,34,33,21,41,28,28,27,20,22,27,28,25,28,22,34,24,21,31]
height=[15,36,38,50,25,31,31,50,24,26,26,29,31,31,30,38,26,33,23,30,31,22]
for i in range(len(length)):
    l = length[i]
    b = breadth[i]
    h = height[i]
    a.append(f'L{l}B{b}H{h}')

In [77]:
grouped_df = merged_df.groupby("Material Description_x").agg({**{'Material Description_x': 'first'}, **{col: 'sum' for col in a}})


In [78]:
grouped_df.head()

,Material Description_x,L61B40H15,L36B25H36,L38B34H38,L54B35H50,L48B34H25,L42B33H31,L31B21H31,L61B41H50,L52B28H24,...,L31B22H31,L38B27H31,L36B28H30,L33B25H38,L32B28H26,L28B22H33,L49B34H23,L34B24H30,L30B21H31,L52B31H22
Material Description_x,,,,,,,,,,,,,,,,,,,,,
AASHIRVAAD SALT - 1KG (25KG OUTER)_FA020,AASHIRVAAD SALT - 1KG (25KG OUTER)_FA020,60177.016535,53099.068423,81472.853915,158850.910924,67334.708356,71026.032281,33002.845440,210914.597859,57354.840903,...,34677.448587,52085.342522,49487.358773,51307.128698,38141.318917,33259.002217,63104.853332,40190.323852,31868.436856,58241.031319
AASHIRVAAD SUPERIOR MP ATTA 1KG_FD2113,AASHIRVAAD SUPERIOR MP ATTA 1KG_FD2113,22874.140872,19882.944617,31951.925640,70018.031507,26220.615732,28464.932527,15226.253885,94991.915864,21017.509140,...,16761.227174,19116.955101,17544.926328,18837.830506,10575.659310,15493.333440,23757.321783,11857.113887,14434.635464,21656.912351
BINGO CHIPS RS.5 CHILLI_FXC60005N,BINGO CHIPS RS.5 CHILLI_FXC60005N,28004.652112,24820.284116,38427.931167,75386.364902,31507.994944,33314.718890,14864.096786,99972.472082,26709.175562,...,15585.514568,24319.116255,23072.421885,23934.381331,17312.851473,14990.589610,29437.686156,18320.326199,14303.914281,27144.169170
BINGO CHIPS RS.5 TOMATO & CHILLI_FXC5000,BINGO CHIPS RS.5 TOMATO & CHILLI_FXC5000,34630.405557,30179.171661,46911.246406,93392.046906,39141.249007,40463.176133,18018.538346,124184.534936,32812.464010,...,18610.420216,29524.236388,27872.155204,29021.457794,20836.306594,18189.779328,36516.410132,22054.302130,17260.185428,33383.315221
BINGO NO RULZ RS.5 CHEESE CURLZ_FXN50000,BINGO NO RULZ RS.5 CHEESE CURLZ_FXN50000,15783.592547,13860.780820,21384.652509,42062.333003,17706.404273,18698.025750,8317.227837,55975.273253,15025.455351,...,8759.488916,13588.840304,12871.906217,13380.077888,9692.857496,8384.878595,16570.114105,10234.907278,8017.631622,15263.517756


In [79]:

# Load the dataset
# data = pd.read_excel('grouped_dry_box_cost.xlsx')
data= grouped_df
# Get the column names starting from the 2nd column
columns = data.columns[1:]

# Find the lowest cost and corresponding column for each row
data['lowest_cost'] = data[columns].idxmin(axis=1)

# Remove the prefix from the column names
data['lowest_cost'] = data['lowest_cost'].str.split('_', expand=True)[0]

# Display the updated dataset



In [80]:
data.head()

,Material Description_x,L61B40H15,L36B25H36,L38B34H38,L54B35H50,L48B34H25,L42B33H31,L31B21H31,L61B41H50,L52B28H24,...,L38B27H31,L36B28H30,L33B25H38,L32B28H26,L28B22H33,L49B34H23,L34B24H30,L30B21H31,L52B31H22,lowest_cost
Material Description_x,,,,,,,,,,,,,,,,,,,,,
AASHIRVAAD SALT - 1KG (25KG OUTER)_FA020,AASHIRVAAD SALT - 1KG (25KG OUTER)_FA020,60177.016535,53099.068423,81472.853915,158850.910924,67334.708356,71026.032281,33002.845440,210914.597859,57354.840903,...,52085.342522,49487.358773,51307.128698,38141.318917,33259.002217,63104.853332,40190.323852,31868.436856,58241.031319,L30B21H31
AASHIRVAAD SUPERIOR MP ATTA 1KG_FD2113,AASHIRVAAD SUPERIOR MP ATTA 1KG_FD2113,22874.140872,19882.944617,31951.925640,70018.031507,26220.615732,28464.932527,15226.253885,94991.915864,21017.509140,...,19116.955101,17544.926328,18837.830506,10575.659310,15493.333440,23757.321783,11857.113887,14434.635464,21656.912351,L40B20H29
BINGO CHIPS RS.5 CHILLI_FXC60005N,BINGO CHIPS RS.5 CHILLI_FXC60005N,28004.652112,24820.284116,38427.931167,75386.364902,31507.994944,33314.718890,14864.096786,99972.472082,26709.175562,...,24319.116255,23072.421885,23934.381331,17312.851473,14990.589610,29437.686156,18320.326199,14303.914281,27144.169170,L30B21H31
BINGO CHIPS RS.5 TOMATO & CHILLI_FXC5000,BINGO CHIPS RS.5 TOMATO & CHILLI_FXC5000,34630.405557,30179.171661,46911.246406,93392.046906,39141.249007,40463.176133,18018.538346,124184.534936,32812.464010,...,29524.236388,27872.155204,29021.457794,20836.306594,18189.779328,36516.410132,22054.302130,17260.185428,33383.315221,L30B21H31
BINGO NO RULZ RS.5 CHEESE CURLZ_FXN50000,BINGO NO RULZ RS.5 CHEESE CURLZ_FXN50000,15783.592547,13860.780820,21384.652509,42062.333003,17706.404273,18698.025750,8317.227837,55975.273253,15025.455351,...,13588.840304,12871.906217,13380.077888,9692.857496,8384.878595,16570.114105,10234.907278,8017.631622,15263.517756,L30B21H31


In [81]:
title=merged_df["Material Description_x"]
d={}
for i in title:
    d[i]=1+d.get(i,0)

In [82]:

# Iterate over the rows of the DataFrame
for index, row in data.iterrows():
    material_description = row["Material Description_x"]
    
    # Check if the material description is present in the dictionary
    if material_description in d:
        count_value = d[material_description]
        
        # Assign the count value to the "count" column
        data.loc[index, "count"] = count_value


In [83]:
data.head()

,Material Description_x,L61B40H15,L36B25H36,L38B34H38,L54B35H50,L48B34H25,L42B33H31,L31B21H31,L61B41H50,L52B28H24,...,L36B28H30,L33B25H38,L32B28H26,L28B22H33,L49B34H23,L34B24H30,L30B21H31,L52B31H22,lowest_cost,count
Material Description_x,,,,,,,,,,,,,,,,,,,,,
AASHIRVAAD SALT - 1KG (25KG OUTER)_FA020,AASHIRVAAD SALT - 1KG (25KG OUTER)_FA020,60177.016535,53099.068423,81472.853915,158850.910924,67334.708356,71026.032281,33002.845440,210914.597859,57354.840903,...,49487.358773,51307.128698,38141.318917,33259.002217,63104.853332,40190.323852,31868.436856,58241.031319,L30B21H31,711.0
AASHIRVAAD SUPERIOR MP ATTA 1KG_FD2113,AASHIRVAAD SUPERIOR MP ATTA 1KG_FD2113,22874.140872,19882.944617,31951.925640,70018.031507,26220.615732,28464.932527,15226.253885,94991.915864,21017.509140,...,17544.926328,18837.830506,10575.659310,15493.333440,23757.321783,11857.113887,14434.635464,21656.912351,L40B20H29,378.0
BINGO CHIPS RS.5 CHILLI_FXC60005N,BINGO CHIPS RS.5 CHILLI_FXC60005N,28004.652112,24820.284116,38427.931167,75386.364902,31507.994944,33314.718890,14864.096786,99972.472082,26709.175562,...,23072.421885,23934.381331,17312.851473,14990.589610,29437.686156,18320.326199,14303.914281,27144.169170,L30B21H31,342.0
BINGO CHIPS RS.5 TOMATO & CHILLI_FXC5000,BINGO CHIPS RS.5 TOMATO & CHILLI_FXC5000,34630.405557,30179.171661,46911.246406,93392.046906,39141.249007,40463.176133,18018.538346,124184.534936,32812.464010,...,27872.155204,29021.457794,20836.306594,18189.779328,36516.410132,22054.302130,17260.185428,33383.315221,L30B21H31,431.0
BINGO NO RULZ RS.5 CHEESE CURLZ_FXN50000,BINGO NO RULZ RS.5 CHEESE CURLZ_FXN50000,15783.592547,13860.780820,21384.652509,42062.333003,17706.404273,18698.025750,8317.227837,55975.273253,15025.455351,...,12871.906217,13380.077888,9692.857496,8384.878595,16570.114105,10234.907278,8017.631622,15263.517756,L30B21H31,190.0


In [84]:
count=data["count"].sum()

In [85]:
# Calculate the sum of the "count" column for each unique value in "lowest_cost"
total_count = data.groupby("lowest_cost")["count"].transform("sum")

# Calculate the percentage based on the sum of "count"
data["percentage"] = total_count/count*100



In [86]:
data

,Material Description_x,L61B40H15,L36B25H36,L38B34H38,L54B35H50,L48B34H25,L42B33H31,L31B21H31,L61B41H50,L52B28H24,...,L33B25H38,L32B28H26,L28B22H33,L49B34H23,L34B24H30,L30B21H31,L52B31H22,lowest_cost,count,percentage
Material Description_x,,,,,,,,,,,,,,,,,,,,,
AASHIRVAAD SALT - 1KG (25KG OUTER)_FA020,AASHIRVAAD SALT - 1KG (25KG OUTER)_FA020,60177.016535,53099.068423,81472.853915,158850.910924,67334.708356,71026.032281,33002.845440,210914.597859,57354.840903,...,51307.128698,38141.318917,33259.002217,63104.853332,40190.323852,31868.436856,58241.031319,L30B21H31,711.0,85.305502
AASHIRVAAD SUPERIOR MP ATTA 1KG_FD2113,AASHIRVAAD SUPERIOR MP ATTA 1KG_FD2113,22874.140872,19882.944617,31951.925640,70018.031507,26220.615732,28464.932527,15226.253885,94991.915864,21017.509140,...,18837.830506,10575.659310,15493.333440,23757.321783,11857.113887,14434.635464,21656.912351,L40B20H29,378.0,1.852124
BINGO CHIPS RS.5 CHILLI_FXC60005N,BINGO CHIPS RS.5 CHILLI_FXC60005N,28004.652112,24820.284116,38427.931167,75386.364902,31507.994944,33314.718890,14864.096786,99972.472082,26709.175562,...,23934.381331,17312.851473,14990.589610,29437.686156,18320.326199,14303.914281,27144.169170,L30B21H31,342.0,85.305502
BINGO CHIPS RS.5 TOMATO & CHILLI_FXC5000,BINGO CHIPS RS.5 TOMATO & CHILLI_FXC5000,34630.405557,30179.171661,46911.246406,93392.046906,39141.249007,40463.176133,18018.538346,124184.534936,32812.464010,...,29021.457794,20836.306594,18189.779328,36516.410132,22054.302130,17260.185428,33383.315221,L30B21H31,431.0,85.305502
BINGO NO RULZ RS.5 CHEESE CURLZ_FXN50000,BINGO NO RULZ RS.5 CHEESE CURLZ_FXN50000,15783.592547,13860.780820,21384.652509,42062.333003,17706.404273,18698.025750,8317.227837,55975.273253,15025.455351,...,13380.077888,9692.857496,8384.878595,16570.114105,10234.907278,8017.631622,15263.517756,L30B21H31,190.0,85.305502
BINGO! CHIPS RS.5 CRM&ON_FXC20005N,BINGO! CHIPS RS.5 CRM&ON_FXC20005N,52715.646860,47481.078226,72903.757786,145274.642509,59644.174631,63251.195898,27637.408357,193468.174960,51668.357206,...,45688.345594,32599.847063,27861.783857,55578.380354,34530.681793,26565.424264,52589.446823,L30B21H31,671.0,85.305502
BINGO! CHIPS RS.5 MASALA_FXC40005N,BINGO! CHIPS RS.5 MASALA_FXC40005N,25841.793816,22490.320785,39202.392147,88031.226507,30545.738270,33082.279319,12911.236965,122038.652918,25674.729082,...,21362.291333,17174.303003,13213.551291,27668.178329,13853.578427,11914.683541,24311.431259,L30B21H31,464.0,85.305502
BINGO! CHIPS RS.5 SALTED_FXC10005N,BINGO! CHIPS RS.5 SALTED_FXC10005N,23847.277869,20906.105539,32473.665185,64534.753539,26947.937617,28441.922546,12533.204299,85825.690661,22796.414989,...,20125.907064,14488.945959,12647.364954,25115.595357,15324.980285,12076.530996,23095.711114,L30B21H31,296.0,85.305502
BINGO! CHIPS RS.5 TOMATO_FXC30005N,BINGO! CHIPS RS.5 TOMATO_FXC30005N,28728.072815,25135.542118,44096.037764,99203.210641,34030.732265,37153.939423,15352.540622,138382.939254,28861.234158,...,23677.815892,19857.002348,15573.957370,30668.774284,15348.665801,14039.226616,27047.860367,L30B21H31,528.0,85.305502


In [87]:
b= data.groupby("lowest_cost")["percentage"].unique()


In [88]:
for lowest_cost, percentages in b.items():
    for percentage in percentages:
        print(lowest_cost, percentage)

L30B21H31 85.30550247439855
L31B21H31 6.849919153314714
L31B22H31 4.233426429516389
L37B27H26 1.759027879856926
L40B20H29 1.8521240629134204


In [ ]:
data.to_excel('final_cost.xlsx', index=False)